In [1]:
from pathlib import Path
if Path.cwd().parent.stem == 't1':
    %cd ..

/home/step/Personal/UCH/2021-sem1/VisionComp/t1


In [2]:
from dataset import ClothingSmall, parse_function, train_preprocess
from train import *
import matplotlib.pyplot as plt

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


# ResNet-34


In [3]:
resnet34_config = \
"""[FASHION-RESNET34]
# Training Related
NUM_EPOCHS = 20
NUM_CLASSES = 19
BATCH_SIZE = 128
SNAPSHOT_STEPS = 500
VALIDATION_STEPS = 100
LEARNING_RATE = 0.003
USE_L2 = True
WEIGHT_DECAY = 1e-4

SNAPSHOT_DIR = /home/step/Personal/UCH/2021-sem1/VisionComp/t1/snapshots/snapshots-r34/

# Dataset Related
DATA_DIR = /home/step/Personal/UCH/2021-sem1/VisionComp/t1/data/clothing-small/
SHUFFLE_SIZE = 10000
CHANNELS = 3
IMAGE_TYPE = SKETCH
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

#for tf_records to use multithreads
USE_MULTITHREADS = True
NUM_THREADS = 10

#CKPFILE is used for fine tunning
#CKPFILE =/home/step/Personal/UCH/2021-sem1/VisionComp/t1/chks
"""

with open("configs/t1_resnet34.config", 'w') as conf:
    conf.write(resnet34_config)

## Configuration and Dataset

In [4]:
config = create_config("FASHION-RESNET34", "configs/t1_resnet34.config")
tfr_files = parse_config(config, mode='train')
datasets = load_dataset(config, tfr_files['train'], tfr_files['test'], 'train')

In [5]:
from dataset import ClothingSmall

process_func = lambda img, label : train_preprocess(img, label, seed=[8, 8])

csDataset = ClothingSmall(data_dir = config.get_data_dir())

csDataset.prepare()

# csDataset.make_ds(parse_function, process_func)

# datasets = {
#     'train' : csDataset.train_ds,
#     'test' : csDataset.test_ds
# }

## Optimizer

In [6]:
cosine_sched = create_scheduler(config)
opt = create_opt('adam', config)

## Create Callbacks

In [7]:
tb_cb, chk_cb = create_cbs(config)

## Model

In [8]:
model = create_model(config, 'resnet-34', csDataset.input_shape, use_mixed=True)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070, compute capability 8.6
Model is Resnet-34
(224, 224, 3)
Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
backbone (ResNetBackbone)    multiple                  21298112  
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  9747      
Total params: 21,307,859
Trainable params: 21,292,627
Non-trainable params: 15,232
_________________________________________________________________


## Run

In [9]:
datasets['train'] = datasets['train'].prefetch(1)
datasets['test'] = datasets['test'].prefetch(1)

In [ ]:
history = run_model('train', model, opt, datasets, config, [chk_cb], [tb_cb])

Epoch 1/20
250/250 [==============================] - 54s 172ms/step - loss: 2.8446 - accuracy: 0.1257 - val_loss: 5.4614 - val_accuracy: 0.1132
Epoch 2/20
250/250 [==============================] - 44s 161ms/step - loss: 2.3950 - accuracy: 0.2192 - val_loss: 2.8414 - val_accuracy: 0.1800
Epoch 3/20
250/250 [==============================] - 45s 162ms/step - loss: 1.9938 - accuracy: 0.3459 - val_loss: 2.2416 - val_accuracy: 0.3153
Epoch 4/20
250/250 [==============================] - 45s 163ms/step - loss: 1.7915 - accuracy: 0.4126 - val_loss: 1.8952 - val_accuracy: 0.3842
Epoch 5/20
250/250 [==============================] - 44s 162ms/step - loss: 1.6420 - accuracy: 0.4566 - val_loss: 1.6769 - val_accuracy: 0.4489
Epoch 6/20
250/250 [==============================] - 45s 163ms/step - loss: 1.5336 - accuracy: 0.4894 - val_loss: 1.7039 - val_accuracy: 0.4447
Epoch 7/20
250/250 [==============================] - 45s 162ms/step - loss: 1.4427 - accuracy: 0.5227 - val_loss: 1.8153 - val_ac

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()